In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
import datetime
import regex
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer


In [81]:
additionalstopwords = ["Deutsche","Deutsch","Deutschen","Deutscher"]
banknames = ["C24 Bank","deutsche ing (diba)","bank c24","C24","Deutsche Bank","ING Deutschland","bank ing-diba","ing diba bank","ing-diba bank","ing.diba","bank ing","ingdiba","ing-diba","ing diba","ing bank","ing  diba","n26 bank","bank n26"
,"N26","Sparda-Bank BW","Sparda-Bank Südwest eG","Volksbank","Commerzbank","dkb bank","bank dkb","DKB","HypoVereinsbank","Hypo","Bunq","bunq bank"
,"Postbank","PSD Braunschweig","PDF Rhein Ruhr","PSD Karlsruhe","PSD berlin brandenburg","Sparda-Bank Hamburg","Sparda-Bank München","sparda- banken","Sparda-Bank Nürnberg"
,"Sparkasse Hannover","Volksbank Mittelhessen","volksbank Berlin","Volksbank Hamburg","Sparkasse Köln bonn",
"sparkasse bank","Sparkasse","sparkasse","Sparda-Bank",
"spardap","sparda bank","sparda banken","psd bank","psd","targobank","revolut","targo bank","tomorrow","vivid money","vividmoney","vivid-money","vivid","fidor bank","fidor","comdirect","norisbank","santander","noris bank","bunq","Fidorbank","fidor bank"
" diba ","dkb"," Diba "," diba","(diba)","diba ","bank deutschen bank","deutsche Bank","deutschen bank"," ING ","Bankbank","bank "]
stop_words = stopwords.words("german") +additionalstopwords 



In [82]:
import glob
datafilepaths = glob.glob("MorerecentData/*")

In [83]:
t = "online banking"
word = "bank."
re.sub(word.lower()," ",t)

'online  ng'

In [86]:
customerevaluations = pd.DataFrame(columns=['company','date','headline','body','rating'])

for files in datafilepaths:
    customerevaluation = pd.read_csv(files)
    customerevaluations = pd.concat([customerevaluations,customerevaluation],ignore_index=True)

customerevaluations["date"] = pd.to_datetime(customerevaluations["date"])
customerevaluations.drop_duplicates(["headline","body","rating"])
customerevaluations["length of review"] = customerevaluations["body"].str.len()

#customerevaluations['body'] = customerevaluations['body'].map(lambda x: x.lower())
customerevaluations["rating"] = pd.to_numeric(customerevaluations['rating'])

In [90]:
summarytable = customerevaluations[["rating","company","length of review"]].groupby(["company"]).describe().round(decimals=2)
summarybyrating= customerevaluations[["rating","length of review"]].groupby("rating").describe().round(decimals=2)
customerevaluations['company'] = customerevaluations['company'].map(lambda x: re.sub('ING Neu2', 'ING', x))
customerevaluations['company'] = customerevaluations['company'].map(lambda x: re.sub('ING Neu', 'ING', x))

with pd.ExcelWriter('summary tables.xlsx') as writer:  
    #summarytable.to_excel(writer, sheet_name='summarybybank')
    summarybyrating.to_excel(writer, sheet_name='summarybyrating')

In [14]:


customerevaluations["originalbody"] = customerevaluations['body']
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('&nt', '', x))



for word in banknames:
    customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(word.lower(),'Bank ', x.lower()))

countrynames = ["Deutschland","Österreich","Schweiz"]
citynames = ["Berlin","Frankfurt"]

for word in countrynames:
    customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(word.lower(), 'Land', x))

for word in citynames:
    customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(word.lower(), 'Stadt', x))
'''
for word in stop_words:
    #customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub(" "+word.lower()+" ", ' ', x))



customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('&', '', x))

'''
customerevaluations['word count'] = customerevaluations['originalbody'].map(lambda x: len(x.strip().split(" ")))






In [15]:
#mois kommenatre ausfiltern
searchfor = ['mois',"Mois","MOIS","keller","Keller","nahuiiiiiiiiiiii","lakakakakslskdnwkskd","einfach aus prinzip nahuyyyyyyy",
"nahui","oh yeah nahui","nahuiiiii","kellwerwärter","schnelllgmarceletkevin","83 jahre alt und nach schlaganfall pflegebedürftig","mühlhshabbshsbshyjabvsuabsbsjabasjss","de54100110012621571864","huens","ich kunde der deutschen bank (odenkirchener str. 43, 41236 mönchengladbach)","bernlöhr66538","inshallah","ist ja ekelhaft","drecksverein.geht den bach runter.","leute geht niemals zur Bank der gröste müll","gröster müll der welt.",".yyyyyyyyyyyyyyyyyy","oaschlöcher","ja sie wollen, dass ich falle lele"
,"gelber oasch nahoi","ya kelb ihr seid einfach lächerlich","jetzt werden önkel gefickt najui","alles neider","richtige lipp lipps die gehören gehauen","wiiixxxxxxxerrrr","erika","wenn koi luscht hen koi zeit hen wedder bleed ish en shineesischer saddelit uf erd stertze dud, konschde deene wadde, un wadde, un wadde, un..., bassiert nix. hen ja fielleischt nerffe","xxxxxxxxxcccccxcxc","einfach aus prinzip nahuyyyyyyy"]
customerevaluations = customerevaluations[~customerevaluations.body.str.contains('|'.join(searchfor))]
customerevaluations['date'] = customerevaluations['date'].apply(lambda a: pd.to_datetime(a).date()) 
customerevaluations = customerevaluations[(customerevaluations["date"]< (datetime.date(2022,4,16)))]
# still mention in research marketing start?

C:\Users\dusti\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


In [16]:
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('\d{1,},\d{1,}.€', 'Betrag', x))
customerevaluations['body'] = customerevaluations['body'].map(lambda x: re.sub('\d{1,}.euro', 'Betrag', x))

In [17]:
customerevaluations.describe()

,rating,length of review,word count
count,39435.000000,39435.000000,39435.000000
mean,3.150298,366.978243,54.861037
std,1.891489,484.785954,73.131875
min,1.000000,6.000000,1.000000
25%,1.000000,89.000000,13.000000
50%,4.000000,209.000000,31.000000
75%,5.000000,453.000000,68.000000
max,5.000000,9941.000000,1299.000000


In [18]:
customerevaluations.groupby("rating").mean()

,length of review,word count
rating,,
1,565.820888,84.970293
2,619.358828,93.013581
3,543.575025,81.676737
4,295.158768,43.776856
5,175.181858,25.848217


In [19]:
# customerevaluations = customerevaluations[customerevaluations["company"]== "Volksbank"]

In [20]:
from bertopic import BERTopic


In [21]:
vectorizer_model = CountVectorizer(stop_words=stop_words)

In [22]:
customerevaluations.reset_index(drop=True, inplace=True)
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = sentence_model.encode(customerevaluations["body"], show_progress_bar=False)

In [78]:

customerevaluations.reset_index(drop=True, inplace=True)
topic_model = BERTopic(language="german",calculate_probabilities=True,top_n_words=10,vectorizer_model=vectorizer_model,diversity=0.3)
topics, probs = topic_model.fit_transform(customerevaluations["body"],embeddings)


In [24]:
#topics_over_time = topic_model.topics_over_time(customerevaluations["body"], topics, customerevaluations["date"], nr_bins=20)
#topics_per_class = topic_model.topics_per_class(customerevaluations["body"],topics,classes= customerevaluations["company"])

In [79]:
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
print(f"{outliers} documents have not been classified")
print(f"The other {topic_freq['Count'].sum() - outliers} documents are {topic_freq['Topic'].shape[0]-1} topics")

24412 documents have not been classified
The other 15023 documents are 266 topics


In [76]:
topics, probs= topic_model.reduce_topics(customerevaluations["body"], topics, nr_topics=80)
topic_model.visualize_topics()

In [67]:
topic_model.visualize_barchart(n_words=10,top_n_topics=60)

In [69]:
customerevaluations["topic"] = topics

In [ ]:
topic_model.get_topic(0)

In [ ]:
t = topic_model.get_topic(1)
len(t)

In [70]:
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
topicsAndTopWords = []
#for i in range(1,len(topic_freq["Topic"])):
for i in range(0,len(topic_freq["Topic"])-1):
    t = topic_model.get_topic(i)
    words = ""

    for w in range(0,len(t)-1):
        words = words + "_" + t[w][0] 
    topicsAndTopWords.append(words)

topicsAndTopWords = pd.DataFrame({"top words": topicsAndTopWords})



In [ ]:
#topic_model.update_topics(customerevaluations["body"], topics, n_gram_range=(1, 1))

In [ ]:
#new_topics, new_probs= topic_model.reduce_topics(customerevaluations["body"], topics, nr_topics=100)


In [ ]:
topic_model.visualize_barchart(n_words=10,top_n_topics=60)

In [ ]:

customerevaluations["reducedtopic"] = new_topics
reducedtopic_freq = topic_model.get_topic_freq()

reducedtopicsAndTopWords = []
#for i in range(1,len(topic_freq["Topic"])):
for i in range(0,len(reducedtopic_freq["Topic"])-1):
    t = topic_model.get_topic(i)
    words = ""

    for w in range(0,len(t)-1):
        words = words + "_" + t[w][0] 
    reducedtopicsAndTopWords.append(words)

reducedtopicsAndTopWords = pd.DataFrame({"top words": reducedtopicsAndTopWords})


In [71]:

with pd.ExcelWriter('15.06NRautoreduced to 100.xlsx') as writer:  
    customerevaluations.to_excel(writer, sheet_name='evaluations',engine="xlsxwriter")
    topicsAndTopWords.to_excel(writer, sheet_name='topics',engine="xlsxwriter")
    #reducedtopicsAndTopWords.to_excel(writer, sheet_name='reduced_topics')

In [ ]:

'''#len(reduction[0]) #to reduce topics then before doing everything with reduced model.
#topic_model.save("Mymodel15topnword")
with pd.ExcelWriter('sentimentanalysis.xlsx') as writer:  
    diccdf.to_excel(writer, sheet_name='Wordlist')
    
'''

In [73]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_heatmap()

In [72]:
topic_model.save("15.06NRautoreduced to 100")

c:\Users\dusti\AppData\Local\Programs\Python\Python37\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [54]:
topic_modeltestreload = BERTopic.load("15.06NRautoreduced to 50")

In [59]:
topic_modeltestreload.visualize_barchart()